In [1]:
import torch
import pandas as pd
import numpy as np
import random

In [2]:
data = pd.read_csv("climate_disease_dataset.csv")
df = data

In [3]:
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-01')
df['country'] = pd.DataFrame(df['country'].astype(str) + '(' + df['region'].astype(str) + ')')
REGION = np.unique(df.region)
DATES = np.unique(df.date)
COUNTRY_NAME = np.unique(df['country'].values)
MONTHS = {1 : 'Jan', 2 : 'Feb', 3 : 'Mar', 4 : 'Apr', 5 : 'May', 6 : 'June',
          7 : 'July', 8 : 'Aug', 9 : 'Sep', 10 : 'Oct', 11 : 'Nov', 12 : 'Dec', 13 : 'Whole Year'}
dic = {}

for name in COUNTRY_NAME:
    dic[name] = df[df.country == name].drop(['healthcare_budget', 'population_density', 'country','region', 'year'], axis=1).reset_index(drop=True)

for reg in REGION:
    dic[reg] = df[df['region'] == reg].groupby('date',as_index=False)[['avg_temp_c', 'uv_index', 'air_quality_index', 'precipitation_mm', 'malaria_cases', 'dengue_cases', 'date', 'month']].mean().reset_index(drop=True)
    dic[reg]['month'] = dic[reg]['month'].astype(int)

In [4]:
df.keys()

Index(['year', 'month', 'country', 'region', 'avg_temp_c', 'precipitation_mm',
       'air_quality_index', 'uv_index', 'malaria_cases', 'dengue_cases',
       'population_density', 'healthcare_budget', 'date'],
      dtype='object')

In [5]:
dic[name].index

RangeIndex(start=0, stop=288, step=1)

In [6]:
dic[name]

,month,avg_temp_c,precipitation_mm,air_quality_index,uv_index,malaria_cases,dengue_cases,date
0,1,29.532837,105.900609,2.289610,12.000000,113,109,2000-01-01
1,2,33.436671,204.743505,11.228808,12.000000,136,94,2000-02-01
2,3,31.987565,153.683992,38.104784,12.000000,134,80,2000-03-01
3,4,29.816317,130.746789,0.000000,12.000000,51,117,2000-04-01
4,5,32.615551,99.971301,0.000000,12.000000,134,126,2000-05-01
...,...,...,...,...,...,...,...,...
283,8,22.786152,179.236451,0.000000,12.000000,16,106,2023-08-01
284,9,21.011076,188.270855,0.000000,11.887276,27,21,2023-09-01
285,10,22.651652,159.423937,1.580432,10.493560,67,71,2023-10-01
286,11,23.837353,60.190065,18.468966,12.000000,30,117,2023-11-01


In [7]:
x = []
y = []
l = 10
a = [[0 for _ in range(4)]for _ in range(l)]
b = [[0 for _ in range(2)]for _ in range(l)]
for key in dic.keys():
    temp_df = dic[key]
    for i in range(temp_df.index.start, temp_df.index.stop, temp_df.index.step):
        a[l - 1] = list([temp_df['avg_temp_c'][i], temp_df['precipitation_mm'][i], temp_df['air_quality_index'][i], temp_df['uv_index'][i]])
        b[l-1] = list([temp_df['malaria_cases'][i], temp_df['dengue_cases'][i]])
        x.append(np.array(a, dtype = np.float32))
        y.append(np.array(b, dtype = np.float32))
        for j in range(l - 1):
            a[j] = a[j + 1]
            b[j] = b[j + 1]

In [8]:
temp_df.index

RangeIndex(start=0, stop=288, step=1)

In [9]:
len(x), len(y)

(36000, 36000)

In [10]:
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(0, len(x), 50) :
    if random.random() <= 0.2:
        x_test+=x[i : i + 50]
        y_test+=y[i : i + 50]
    else :
        x_train+=x[i : i + 50]
        y_train+=y[i : i + 50]     

In [11]:
len(x_train)

24550

In [12]:
len(x_test)

11450

In [13]:
x_train = np.array(x_train, dtype = np.float32)
x_test = np.array(x_test, dtype = np.float32)
y_train = np.array(y_train, dtype = np.float32)
y_test = np.array(y_test, dtype = np.float32)

In [14]:
len(y_test)

11450

In [15]:
d = {
    'x_train' : x_train,
    'x_test' : x_test,
    'y_train' : y_train,
    'y_test' : y_test,
}

In [16]:
np.savez('Data', **d)

In [17]:
x_train[500:502]

array([[[ 16.646215 , 328.8227   ,  10.488923 ,  12.       ],
        [ 14.859339 , 251.1813   ,  15.473769 ,  11.25089  ],
        [ 18.954456 , 292.88034  ,  20.1405   ,  12.       ],
        [ 24.387983 , 203.4421   ,  69.88813  ,   7.1318774],
        [ 26.176033 , 247.02904  ,  75.479866 ,  11.458607 ],
        [ 25.883562 , 211.64346  ,   0.       ,  11.306276 ],
        [ 28.383398 , 270.5102   ,  32.744167 ,  10.234283 ],
        [ 23.340456 , 209.03441  ,   0.       ,  12.       ],
        [ 18.442076 , 314.5988   ,  33.01833  ,  12.       ],
        [ 16.761812 , 200.92487  ,  50.92178  ,  12.       ]],

       [[ 14.859339 , 251.1813   ,  15.473769 ,  11.25089  ],
        [ 18.954456 , 292.88034  ,  20.1405   ,  12.       ],
        [ 24.387983 , 203.4421   ,  69.88813  ,   7.1318774],
        [ 26.176033 , 247.02904  ,  75.479866 ,  11.458607 ],
        [ 25.883562 , 211.64346  ,   0.       ,  11.306276 ],
        [ 28.383398 , 270.5102   ,  32.744167 ,  10.234283 ],
      

In [18]:
x_train[1]

array([[ 13.380206 , 291.8719   ,   0.       ,  10.527265 ],
       [ 11.724961 , 289.44092  ,  45.684746 ,  12.       ],
       [  9.738345 , 281.57803  ,  12.078513 ,  12.       ],
       [ 11.740838 , 186.4619   ,  35.350967 ,   9.171999 ],
       [ 13.1386385, 229.28162  ,   9.323229 ,  12.       ],
       [ 18.257786 , 146.97997  ,   2.171863 ,   7.7657156],
       [ 22.88743  , 226.37204  ,  12.335747 ,  11.935264 ],
       [ 27.917664 , 136.07204  ,  59.055386 ,   7.8727098],
       [ 29.301603 , 180.8215   ,   8.506149 ,  10.949083 ],
       [ 25.170374 , 109.33289  ,  16.417788 ,  11.162393 ]],
      dtype=float32)

In [19]:
y_train

array([[[ 92.      , 108.      ],
        [111.      ,  90.      ],
        [  0.      ,  49.      ],
        ...,
        [127.      ,  43.      ],
        [  0.      ,  67.      ],
        [102.      ,  54.      ]],

       [[111.      ,  90.      ],
        [  0.      ,  49.      ],
        [ 18.      ,  14.      ],
        ...,
        [  0.      ,  67.      ],
        [102.      ,  54.      ],
        [  0.      ,  42.      ]],

       [[  0.      ,  49.      ],
        [ 18.      ,  14.      ],
        [  0.      ,  64.      ],
        ...,
        [102.      ,  54.      ],
        [  0.      ,  42.      ],
        [ 19.      , 100.      ]],

       ...,

       [[ 77.066666,  89.73333 ],
        [ 86.03333 ,  98.26667 ],
        [ 89.76667 , 100.566666],
        ...,
        [ 63.7     ,  67.833336],
        [ 57.2     ,  64.      ],
        [ 68.566666,  68.76667 ]],

       [[ 86.03333 ,  98.26667 ],
        [ 89.76667 , 100.566666],
        [ 94.36667 , 106.53333 ],
        .